<a href="https://colab.research.google.com/github/emanfatima20/AI-Powered-Path-Reccomender/blob/main/RESNET_QUANTIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, shutil


os.makedirs("/root/.kaggle", exist_ok=True)


shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

!chmod 600 /root/.kaggle/kaggle.json


**IMPORTING DATASET**

In [ ]:
import kagglehub


path = kagglehub.dataset_download("utkarshsaxenadn/fruits-classification")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'fruits-classification' dataset.
Path to dataset files: /kaggle/input/fruits-classification


In [ ]:
import shutil
import os


original_path = "/kaggle/input/fruits-classification"


dest_path = "/content/fruits-classification"


shutil.copytree(original_path, dest_path)

print("Dataset is now accessible at:", dest_path)


Dataset is now accessible at: /content/fruits-classification


**CREATING BATCHES FOR DATASET**

In [ ]:
import tensorflow as tf
train_dir = "/content/fruits-classification/Fruits Classification/train"
test_dir  = "/content/fruits-classification/Fruits Classification/test"

img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)


from tensorflow.keras.applications.resnet50 import preprocess_input
def preprocess(images, labels):
    return preprocess_input(images), labels

train_ds = train_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)


Found 9700 files belonging to 5 classes.
Found 100 files belonging to 5 classes.


**IMPORTING RESNET AND FINE TUNING IT ON MY MODEL TO IMPROVE ACCURACY ON MY DATASET**

In [ ]:
from tensorflow.keras.layers import  Dense, Dropout, Input
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_tensor=Input(shape=(224,224,3))
)

# Freeze all CNN layers (Conv2D + pooling)
base_model.trainable = False


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential


from tensorflow.keras.layers import Dense, Dropout, Input, Flatten



model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint_cb = ModelCheckpoint(
    "resnet_fruit_model_epoch_{epoch:02d}.keras",
    save_best_only=False,
    save_weights_only=False,
    verbose=1
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=[checkpoint_cb]
)


Epoch 1/10
304/304 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.7173 - loss: 3.6580
Epoch 1: saving model to resnet_fruit_model_epoch_01.keras
304/304 ━━━━━━━━━━━━━━━━━━━━ 54s 132ms/step - accuracy: 0.7176 - loss: 3.6503 - val_accuracy: 0.8400 - val_loss: 0.5089
Epoch 2/10
303/304 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.9033 - loss: 0.2865
Epoch 2: saving model to resnet_fruit_model_epoch_02.keras
304/304 ━━━━━━━━━━━━━━━━━━━━ 28s 92ms/step - accuracy: 0.9033 - loss: 0.2865 - val_accuracy: 0.8700 - val_loss: 0.3898
Epoch 3/10
303/304 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.9468 - loss: 0.1608
Epoch 3: saving model to resnet_fruit_model_epoch_03.keras
304/304 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.9468 - loss: 0.1608 - val_accuracy: 0.8800 - val_loss: 0.5215
Epoch 4/10
303/304 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.9594 - loss: 0.1184
Epoch 4: saving model to resnet_fruit_model_epoch_04.keras
304/304 ━━━━━━━━━━━━━━━━━━━━ 41s 92ms/step - accuracy: 0.

In [ ]:
from keras.models import load_model

final_model = load_model("resnet_fruit_model_epoch_10.keras")


In [ ]:

loss, acc = final_model.evaluate(test_ds, verbose=0)
print(f"Original Model Accuracy: {acc*100:.2f}%")

Original Model Accuracy: 85.00%


In [ ]:
import os

keras_model_file = "resnet_fruit_model_epoch_10.keras"
size_bytes = os.path.getsize(keras_model_file)
size_mb = size_bytes / (1024 * 1024)
print(f"Keras model size: {size_mb:.2f} MB")

Keras model size: 237.62 MB


CONCLUSION:FOR ORIGINLAL MODEL THE SIZE IS 265 MB AND ACCURACY IS 85%

**POST TRAINING QUANTIZATION**

In [ ]:
import tensorflow as tf
import numpy as np
import os
from keras.models import load_model


final_model = load_model("resnet_fruit_model_epoch_10.keras")



def representative_data_gen():
    for images, _ in train_ds.take(20):
        images = tf.cast(images, tf.float32)
        yield [images[:8]]

converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]


converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8


tflite_model_int8 = converter.convert()


tflite_file = "resnet_fruit_model_int8.tflite"
with open(tflite_file, "wb") as f:
    f.write(tflite_model_int8)
print(f"Full int8 TFLite model saved: {tflite_file}")


size_mb = os.path.getsize(tflite_file) / (1024 * 1024)
print(f"TFLite model size: {size_mb:.2f} MB")


interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


accuracy = 0
total = 0

for images, labels in test_ds:

    images_np = images.numpy()


    scale, zero_point = input_details[0]['quantization']
    images_int8 = images_np / scale + zero_point
    images_int8 = images_int8.astype(np.int8)


    interpreter.resize_tensor_input(input_details[0]['index'], images_int8.shape)
    interpreter.allocate_tensors()
    interpreter.set_tensor(input_details[0]['index'], images_int8)
    interpreter.invoke()


    predictions = interpreter.get_tensor(output_details[0]['index'])
    scale_out, zero_point_out = output_details[0]['quantization']
    predictions_float = (predictions.astype(np.float32) - zero_point_out) * scale_out
    predicted_labels = np.argmax(predictions_float, axis=1)

    accuracy += np.sum(predicted_labels == labels.numpy())
    total += len(labels)

tflite_acc = accuracy / total
print(f"Full int8 TFLite Model Accuracy: {tflite_acc*100:.2f}%")


Saved artifact at '/tmp/tmpyr03_n1c'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  137927314159760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314158992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314160144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314159184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314159376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314158416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314159568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314158800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314441424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137927314441616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1379273141605

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Full int8 TFLite model saved: resnet_fruit_model_int8.tflite
TFLite model size: 35.37 MB


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Full int8 TFLite Model Accuracy: 84.00%


In [ ]:

tflite_file = "resnet_fruit_model_int8.tflite"

interpreter = tf.lite.Interpreter(model_path=tflite_file)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [ ]:
import os

tflite_file = "resnet_fruit_model_int8.tflite"


size_bytes = os.path.getsize(tflite_file)

size_mb = size_bytes / (1024 * 1024)

print(f"TFLite model size: {size_mb:.2f} MB")


TFLite model size: 35.37 MB


In [ ]:
import os
from google.colab import files


tflite_file = "resnet_fruit_model_int8.tflite"
keras_model_file = "resnet_fruit_model_epoch_10.keras"


size_bytes = os.path.getsize(keras_model_file)
size_mb = size_bytes / (1024 * 1024)
print(f"Keras model size: {size_mb:.2f} MB")

size_bytes = os.path.getsize(tflite_file)
size_mb = size_bytes / (1024 * 1024)
print(f"TFLite model size: {size_mb:.2f} MB")


files.download(keras_model_file)
files.download(tflite_file)


Keras model size: 237.62 MB
TFLite model size: 35.37 MB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CONCLUSION: QUANTIZED MODEL HAS SIZE OF 35 MB NOW AND ACCURACY IS 84%(ONLY 1% DROP IN ACCURACY)